# Training a basic setting with a Deep Q Network (DQN) #

Import statements

In [22]:
import json
import os

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex=True)
%matplotlib inline

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam, SGD

In [3]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory  # For experience replay!

In [15]:
from gym_environment_ncml import *
from learning import *

Useful numbers

In [5]:
MILLION = 1000000
HTHOUSAND = 100000
THOUSAND = 1000

## 1. Create environment ##

In [6]:
env = GridworldMultiAgentv25()

/Users/copernico/opt/anaconda3/envs/RL/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
states = env.observation_space.shape[0]
actions = env.action_space.n

In [8]:
states, actions

(10, 25)

## 2. Create a Deep Learning Model with Keras ##

In [9]:
model = build_model(states, actions, [32, 16], ['relu', 'relu'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 25)                425       
Total params: 1,305
Trainable params: 1,305
Non-trainable params: 0
_________________________________________________________________


## 3. Build Agent with Keras-RL ##

In [11]:
dqn = build_agent(model, actions, 0.01, EpsGreedyQPolicy(), 50000)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
# dqn.compile(Adam(lr=1e-2), metrics=['mse'])

In [17]:
name = 'dqn25_5b5_3236_adam_lr0.001_tmu0.01_ml50K_ns5M_eps0.1'

In [12]:
history = dqn.fit(env, nb_steps=5*MILLION, visualize=False, verbose=1)

Training for 5000000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 64s 6ms/step - reward: -0.6472
200 episodes - episode_reward: -32.360 [-58.000, 26.000] - loss: 1.490 - mae: 2.423 - mean_q: -1.613

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 71s 7ms/step - reward: -0.1434
200 episodes - episode_reward: -7.170 [-54.000, 100.000] - loss: 2.356 - mae: 3.513 - mean_q: 1.236

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 78s 8ms/step - reward: 0.1508
200 episodes - episode_reward: 7.540 [-50.000, 112.000] - loss: 5.485 - mae: 13.538 - mean_q: 15.368

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 75s 8ms/step - reward: 0.6254
200 episodes - episode_reward: 31.270 [-60.000, 132.000] - loss: 10.715 - mae: 24.133 - mean_q: 26.708

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 72s 7ms/step - reward: 0.7732
20

10000/10000 [==============================] - 56s 6ms/step - reward: 1.4556
200 episodes - episode_reward: 72.780 [-52.000, 174.000] - loss: 91.400 - mae: 86.022 - mean_q: 93.887

Interval 40 (390000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 1.6252
200 episodes - episode_reward: 81.260 [-36.000, 184.000] - loss: 90.340 - mae: 86.119 - mean_q: 93.914

Interval 41 (400000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 1.5870
200 episodes - episode_reward: 79.350 [-50.000, 188.000] - loss: 94.505 - mae: 88.172 - mean_q: 96.125

Interval 42 (410000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 1.5538
200 episodes - episode_reward: 77.690 [-50.000, 148.000] - loss: 92.616 - mae: 88.335 - mean_q: 96.202

Interval 43 (420000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 1.4294
200 episodes - episode_reward: 71.470 [-42.000, 170

10000/10000 [==============================] - 67s 7ms/step - reward: 1.6376
200 episodes - episode_reward: 81.880 [-54.000, 176.000] - loss: 91.268 - mae: 86.514 - mean_q: 94.350

Interval 78 (770000 steps performed)
10000/10000 [==============================] - 67s 7ms/step - reward: 1.5720
200 episodes - episode_reward: 78.600 [-50.000, 180.000] - loss: 91.824 - mae: 86.491 - mean_q: 94.227

Interval 79 (780000 steps performed)
10000/10000 [==============================] - 66s 7ms/step - reward: 1.6360
200 episodes - episode_reward: 81.800 [-34.000, 184.000] - loss: 90.039 - mae: 85.583 - mean_q: 93.144

Interval 80 (790000 steps performed)
10000/10000 [==============================] - 65s 6ms/step - reward: 1.4014
200 episodes - episode_reward: 70.070 [-54.000, 178.000] - loss: 90.460 - mae: 85.775 - mean_q: 93.308

Interval 81 (800000 steps performed)
10000/10000 [==============================] - 62s 6ms/step - reward: 1.5922
200 episodes - episode_reward: 79.610 [-40.000, 178

10000/10000 [==============================] - 68s 7ms/step - reward: 1.7302
200 episodes - episode_reward: 86.510 [-36.000, 202.000] - loss: 95.987 - mae: 88.125 - mean_q: 95.484

Interval 116 (1150000 steps performed)
10000/10000 [==============================] - 65s 6ms/step - reward: 1.8530
200 episodes - episode_reward: 92.650 [-40.000, 186.000] - loss: 98.425 - mae: 89.776 - mean_q: 97.409

Interval 117 (1160000 steps performed)
10000/10000 [==============================] - 55s 6ms/step - reward: 1.8010
200 episodes - episode_reward: 90.050 [-50.000, 178.000] - loss: 98.658 - mae: 89.772 - mean_q: 97.376

Interval 118 (1170000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 1.9672
200 episodes - episode_reward: 98.360 [-50.000, 220.000] - loss: 98.840 - mae: 89.670 - mean_q: 97.536

Interval 119 (1180000 steps performed)
10000/10000 [==============================] - 78s 8ms/step - reward: 1.9166
200 episodes - episode_reward: 95.830 [-34.

10000/10000 [==============================] - 54s 5ms/step - reward: 2.1288
200 episodes - episode_reward: 106.440 [-50.000, 214.000] - loss: 148.418 - mae: 111.413 - mean_q: 120.964

Interval 153 (1520000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 2.2778
200 episodes - episode_reward: 113.890 [-30.000, 212.000] - loss: 145.538 - mae: 109.174 - mean_q: 118.405

Interval 154 (1530000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 2.3292
200 episodes - episode_reward: 116.460 [-30.000, 256.000] - loss: 137.034 - mae: 105.642 - mean_q: 114.735

Interval 155 (1540000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 2.4734
200 episodes - episode_reward: 123.670 [-8.000, 224.000] - loss: 139.053 - mae: 107.181 - mean_q: 116.410

Interval 156 (1550000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 2.3852
200 episodes - episode_rewar

...

10000/10000 [==============================] - 52s 5ms/step - reward: 2.6402
200 episodes - episode_reward: 132.010 [2.000, 230.000] - loss: 156.017 - mae: 114.541 - mean_q: 124.186

Interval 338 (3370000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.7328
200 episodes - episode_reward: 136.640 [12.000, 272.000] - loss: 155.796 - mae: 115.041 - mean_q: 124.669

Interval 339 (3380000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.7370
200 episodes - episode_reward: 136.850 [-24.000, 286.000] - loss: 155.550 - mae: 114.939 - mean_q: 124.312

Interval 340 (3390000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.6940
200 episodes - episode_reward: 134.700 [-2.000, 252.000] - loss: 154.940 - mae: 113.597 - mean_q: 123.016

Interval 341 (3400000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.7226
200 episodes - episode_reward: 

10000/10000 [==============================] - 52s 5ms/step - reward: 2.6728
200 episodes - episode_reward: 133.640 [-48.000, 244.000] - loss: 160.525 - mae: 116.808 - mean_q: 126.577

Interval 375 (3740000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.6286
200 episodes - episode_reward: 131.430 [-40.000, 226.000] - loss: 155.887 - mae: 115.035 - mean_q: 124.660

Interval 376 (3750000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.6426
200 episodes - episode_reward: 132.130 [-36.000, 250.000] - loss: 157.268 - mae: 114.586 - mean_q: 124.206

Interval 377 (3760000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.5262
200 episodes - episode_reward: 126.310 [-40.000, 230.000] - loss: 153.209 - mae: 111.499 - mean_q: 120.807

Interval 378 (3770000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.6096
200 episodes - episode_rewa

10000/10000 [==============================] - 52s 5ms/step - reward: 2.6196
200 episodes - episode_reward: 130.980 [-12.000, 256.000] - loss: 137.790 - mae: 107.126 - mean_q: 115.704

Interval 412 (4110000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.5370
200 episodes - episode_reward: 126.850 [0.000, 246.000] - loss: 136.668 - mae: 107.130 - mean_q: 115.591

Interval 413 (4120000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.6036
200 episodes - episode_reward: 130.180 [6.000, 238.000] - loss: 138.724 - mae: 107.705 - mean_q: 116.212

Interval 414 (4130000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.5934
200 episodes - episode_reward: 129.670 [-8.000, 266.000] - loss: 140.693 - mae: 108.757 - mean_q: 117.377

Interval 415 (4140000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.7134
200 episodes - episode_reward: 1

10000/10000 [==============================] - 52s 5ms/step - reward: 1.7630
200 episodes - episode_reward: 88.150 [-50.000, 232.000] - loss: 168.413 - mae: 120.655 - mean_q: 129.850

Interval 449 (4480000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.3992
200 episodes - episode_reward: 119.960 [-30.000, 248.000] - loss: 143.629 - mae: 109.790 - mean_q: 118.452

Interval 450 (4490000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.3642
200 episodes - episode_reward: 118.210 [-58.000, 244.000] - loss: 133.005 - mae: 105.813 - mean_q: 114.321

Interval 451 (4500000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.5222
200 episodes - episode_reward: 126.110 [10.000, 230.000] - loss: 124.975 - mae: 102.167 - mean_q: 110.433

Interval 452 (4510000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.4774
200 episodes - episode_reward

10000/10000 [==============================] - 52s 5ms/step - reward: 2.8556
200 episodes - episode_reward: 142.780 [-50.000, 246.000] - loss: 139.904 - mae: 107.288 - mean_q: 116.464

Interval 486 (4850000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 3.0362
200 episodes - episode_reward: 151.810 [34.000, 274.000] - loss: 146.263 - mae: 109.165 - mean_q: 118.502

Interval 487 (4860000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.9036
200 episodes - episode_reward: 145.180 [-12.000, 250.000] - loss: 151.269 - mae: 112.686 - mean_q: 122.281

Interval 488 (4870000 steps performed)
10000/10000 [==============================] - 52s 5ms/step - reward: 2.6512
200 episodes - episode_reward: 132.560 [-40.000, 268.000] - loss: 160.915 - mae: 116.622 - mean_q: 126.323

Interval 489 (4880000 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: 2.8400
200 episodes - episode_rewar

In [26]:
data = history.history
data['episode_reward'] = [float(v) for v in data['episode_reward']]
data['nb_episode_steps'] = [int(v) for v in data['nb_episode_steps']]
data['nb_steps'] = [int(v) for v in data['nb_steps']]

In [27]:
data

{'episode_reward': [-50.0,
  -50.0,
  -36.0,
  -40.0,
  -40.0,
  -40.0,
  -50.0,
  -50.0,
  -42.0,
  -26.0,
  -46.0,
  -40.0,
  -52.0,
  -20.0,
  -40.0,
  -6.0,
  20.0,
  -50.0,
  -12.0,
  -40.0,
  -50.0,
  -20.0,
  -50.0,
  -40.0,
  -32.0,
  -40.0,
  -6.0,
  -46.0,
  -46.0,
  -50.0,
  -22.0,
  -6.0,
  -26.0,
  -40.0,
  -32.0,
  -40.0,
  -26.0,
  -40.0,
  -40.0,
  -46.0,
  -32.0,
  -2.0,
  -20.0,
  -30.0,
  -30.0,
  -36.0,
  -30.0,
  -50.0,
  -40.0,
  14.0,
  -30.0,
  -50.0,
  -54.0,
  -16.0,
  -26.0,
  -46.0,
  -8.0,
  -50.0,
  -22.0,
  -50.0,
  -40.0,
  -44.0,
  -46.0,
  -8.0,
  -24.0,
  -50.0,
  18.0,
  -28.0,
  -10.0,
  -50.0,
  10.0,
  -50.0,
  -50.0,
  -32.0,
  -36.0,
  -52.0,
  0.0,
  -40.0,
  -40.0,
  -42.0,
  -40.0,
  -40.0,
  -40.0,
  -40.0,
  -26.0,
  -50.0,
  -50.0,
  -50.0,
  -30.0,
  -50.0,
  -50.0,
  -20.0,
  -26.0,
  -50.0,
  -30.0,
  -50.0,
  -2.0,
  -38.0,
  -50.0,
  -36.0,
  -36.0,
  20.0,
  -50.0,
  -20.0,
  -50.0,
  -40.0,
  -50.0,
  -8.0,
  -2.0,
  -40.0,
  -40.0,

In [28]:
os.mkdir('agents/{}'.format(name))  # If the directory does not exist we cannot write the file
with open(get_training_path(name), 'w') as f:
    json.dump(data, f)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))

ax.plot(history.history['nb_steps'], history.history['episode_reward'])

fig.tight_layout()
plt.show()

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Save agent to memory

In [29]:
dqn.save_weights(get_agent_path(name), overwrite=True)

## 4. Reloading training from Memory ##